In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Get rid of warnings
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [4]:
# Check if there are null/na values to deal with
filename = "rotk_playable.csv"

pd_dataset = pd.read_csv(filename)
print("General Data Check:")
print(pd_dataset.head())
print("Number of Rows:", len(pd_dataset))

General Data Check:
           Name Kingdom  Age at Death  LEA  STR  INT  POL  CHA
0  Bao Sanniang     Shu            50   75   85   49   46   74
1     Cai Wenji     Wei            81   21   16   78   80   85
2       Cao Cao     Wei            65   99   73   93   97   96
3        Cao Pi     Wei            39   70   71   81   85   73
4       Cao Ren     Wei            55   92   86   59   51   76
Number of Rows: 93
NA/NULL count:
Name            0
Kingdom         0
Age at Death    0
LEA             0
STR             0
INT             0
POL             0
CHA             0
dtype: int64


In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

for i in range(15, 1, -1):
    pca = PCA(n_components=2)
    pd_pca = pca.fit_transform(pd_dataset_clean)    

    # Approximate explained variance ratio    
    print(f"PCA with {i} components variance:", pca.explained_variance_ratio_.sum())        

    # Compute pairwise distances before and after projection
    orig_dist = euclidean_distances(pd_dataset_clean)
    proj_dist = euclidean_distances(pd_pca)
    
    # Compare distance matrices
    correlation = np.corrcoef(orig_dist.ravel(), proj_dist.ravel())[0, 1]
    print(f"Correlation between original and projected distances: {correlation:.8f}")